In [ ]:
#imports
%matplotlib inline
import json
import pandas as pd

import matplotlib.pyplot as plt

# Datapreparation

In [ ]:
PATH_TO_REFERENCES = './data/raw/references.json'
PATH_TO_CROWD_DATA = './data/raw/anonymized_project.json'

In [ ]:
# load references
references = pd.read_json(PATH_TO_REFERENCES,  orient='index')
references  = references.reset_index(names='img')
references.tail(3)

In [ ]:
with open(PATH_TO_CROWD_DATA, 'r') as f:
    crowddata_json = json.load(f)
crowddata = pd.DataFrame.from_dict(crowddata_json['results']['root_node']['results'],orient='index',)
crowddata.head(2)

In [ ]:
crowddata['gui_type'].unique()

In [ ]:
crowddataframe = pd.DataFrame()
# loop over 'results'
for key in crowddata_json['results']['root_node']['results'].keys():
   if crowddataframe is None:
      crowddataframe = pd.json_normalize(crowddata_json['results']['root_node']['results'][key]['results'])
   else:
      normlist = pd.json_normalize(crowddata_json['results']['root_node']['results'][key]['results'])
      crowddataframe = pd.concat([crowddataframe,normlist])

In [ ]:
crowddataframe.to_csv("./data/project.csv", index=False)

In [ ]:
crowddataframe.head(3)

# TASK 1 - Gather insights about the annotators

## TASK 1a

In [ ]:
print(crowddataframe['user.vendor_id'].unique(), len(crowddataframe['user.vendor_id'].unique()))
print(crowddataframe['user.vendor_user_id'].unique(), len(crowddataframe['user.vendor_user_id'].unique()))

22

a) 22 Annotator (1 Vendor)

## TASK 1b

In [ ]:
crowddataframe['task_output.duration_ms'].min(), crowddataframe['task_output.duration_ms'].max(), crowddataframe['task_output.duration_ms'].mean()

In [ ]:
m = crowddataframe['task_output.duration_ms'] > 0
len(crowddataframe), len(crowddataframe[m]),len(crowddataframe[~m])

In [ ]:
valid_crowddataframe = crowddataframe[m]
valid_crowddataframe['task_output.duration_ms'].min(), valid_crowddataframe['task_output.duration_ms'].max(), valid_crowddataframe['task_output.duration_ms'].mean()

In [ ]:
fig, ax = plt.subplots()
HIST_BINS = 1000
_, _, bar_container = ax.hist(valid_crowddataframe['task_output.duration_ms'], HIST_BINS)
plt.show()

In [ ]:
quant = valid_crowddataframe['task_output.duration_ms'].quantile([0.05,0.95])

In [ ]:
len(valid_crowddataframe['task_output.duration_ms'])

In [ ]:
fig, ax = plt.subplots()
HIST_BINS = 1000
_, _, bar_container = ax.hist(valid_crowddataframe['task_output.duration_ms'], HIST_BINS)
ax.set_xlim(xmax=quant[0.95])
ax.set_xlim(xmin=quant[0.05])
ax.set_xlabel('Sections (quantile 0.05 0.95)')
ax.set_ylabel('Frequency')
#ax.set_title('Histogram')
m = valid_crowddataframe['task_output.duration_ms'].mean()
plt.axvline(m, color='r') # vertical
plt.show()

1b) 

Alle Datenpunkte
* Min 10
* Max 42398
* Mean 1289.9


Negative Datenpunkte aussortiert (Anz:6)
* Min -99999
* Max 42398
* Mean 1284.3

## Task 1c

In [ ]:
anz_annotation_by_user = crowddataframe.groupby('user.vendor_user_id').agg({'user.vendor_user_id': 'count'})
anz_annotation_by_user = anz_annotation_by_user.rename(columns={"user.vendor_user_id": "count"}).sort_values(by=['count'])
print(anz_annotation_by_user)

In [ ]:
annotator_nr = anz_annotation_by_user.reset_index()['user.vendor_user_id'].apply(lambda x: x[-2:])

fig, ax = plt.subplots()
ax.scatter(annotator_nr, anz_annotation_by_user['count'])

ax.set_xlabel('22 Annotators')
ax.set_ylabel('Annotation Count')

## Task 1d

In [ ]:
annotationdata = crowddataframe[['root_input.image_url',
               'user.vendor_user_id',
               'task_output.corrupt_data',
               'task_output.cant_solve',
               'task_output.answer']].copy() # settingwithcopywarning
annotationdata['answer_y'] = annotationdata['task_output.answer']=='yes'
annotationdata.head(3)

In [ ]:
upper_limit = 8
lower_limit = 2

In [ ]:
answers_by_image = annotationdata.groupby(['root_input.image_url'])['answer_y'].sum().reset_index()
disagree_images = answers_by_image[(answers_by_image['answer_y'] < upper_limit) & (lower_limit  < answers_by_image['answer_y'])]['root_input.image_url'].to_list()
print(len(disagree_images))
disagree_images[-1]

## Task 2

In [ ]:
annotationdata = crowddataframe[['root_input.image_url',
               'user.vendor_user_id',
               'task_output.corrupt_data',
               'task_output.cant_solve',
               'task_output.answer',
               'task_output.duration_ms']].copy() # settingwithcopywarning
annotationdata['answer_y'] = annotationdata['task_output.answer']=='yes'
annotationdata.columns

In [ ]:
# number of 'corrupted', 'cant solve', 'both'
len(annotationdata[annotationdata['task_output.corrupt_data']]) , len(annotationdata[annotationdata['task_output.cant_solve']]), len(annotationdata[annotationdata['task_output.corrupt_data'] & annotationdata['task_output.cant_solve']])

# -> 4, 17 ,0

In [ ]:
# Really corrupted? (corrupt_data > 1)
count_cant_solve = annotationdata[annotationdata['task_output.corrupt_data']].groupby('root_input.image_url')['task_output.corrupt_data'].count().reset_index()
len(count_cant_solve[count_cant_solve['task_output.corrupt_data']> 1])

# -> 0 No

In [ ]:
# Really cant solve? (cant'solve > 1)
count_cant_Solve = annotationdata[annotationdata['task_output.cant_solve']].groupby('root_input.image_url')['task_output.cant_solve'].count().reset_index()
len(count_cant_Solve[count_cant_Solve['task_output.cant_solve']> 1])
# -> 0 No

In [ ]:
# check 'problems' with duration # mean duration: ~1300
annotationdata[annotationdata['task_output.corrupt_data'] | annotationdata['task_output.cant_solve']]['task_output.duration_ms'].hist()

# -> faster then mean buth not to fast

In [ ]:
# problems & disagree
problem_annotations = annotationdata[annotationdata['task_output.corrupt_data'] | annotationdata['task_output.cant_solve']]
len(problem_annotations[problem_annotations['root_input.image_url'].isin(disagree_images)])

In [ ]:
# Annotators corrupted
user_corrupt_data = annotationdata[annotationdata['task_output.corrupt_data']]['user.vendor_user_id'].unique().tolist()
user_corrupt_data

In [ ]:
# Annotators can't solve
user_cant_solve = annotationdata[annotationdata['task_output.cant_solve']]['user.vendor_user_id'].unique().tolist()
user_cant_solve

In [ ]:
# Annotators both
[i for i in user_cant_solve if i in user_corrupt_data]

In [ ]:
user_with_problems = user_cant_solve.copy()
user_with_problems.extend(user_corrupt_data)
user_with_problems = list(set(user_with_problems)) 
user_with_problems

In [ ]:
# check % yes/no  
annotation_cup = annotationdata[annotationdata['user.vendor_user_id'].isin(user_with_problems)].groupby('user.vendor_user_id')['answer_y'].agg(['count', 'sum']).reset_index()
annotation_cup['%'] = annotation_cup['sum'] / annotation_cup['count']
annotation_cup
# -> ~50/50 expected -> ok

# Task 3

In [ ]:
# feature disagree
crowddataframe['disagree'] = crowddataframe['root_input.image_url'].isin(disagree_images)

# create crowd dataset
crowd_data_set = crowddataframe[['root_input.image_url',
               'disagree',]
              ].copy() # settingwithcopywarning

crowd_data_set = crowd_data_set.drop_duplicates()

In [ ]:
answer_y = answers_by_image.set_index('root_input.image_url')


# (5 -> no security, 8-2)
def converter(img_name):
    value = answer_y.loc[img_name,'answer_y']
    if value > 5:
        return True
    elif 5 > value: 
        return False
    else:
        return -1

In [ ]:
crowd_data_set['bicycle?'] = crowd_data_set['root_input.image_url'].apply(converter)

In [ ]:
c_is_bicycle = len(crowd_data_set[crowd_data_set['bicycle?']==True]) 
c_is_not_bycle = len(crowd_data_set[crowd_data_set['bicycle?']==False]) 
c_unkown_bycle = len(crowd_data_set[crowd_data_set['bicycle?']==-1])

In [ ]:
# 
c_is_bicycle = len(crowd_data_set[crowd_data_set['bicycle?']==True]) 
c_is_not_bycle = len(crowd_data_set[crowd_data_set['bicycle?']==False]) 
c_unkown_bycle = len(crowd_data_set[crowd_data_set['bicycle?']==-1])

c_is_bicycle, c_is_not_bycle, c_unkown_bycle

In [ ]:
fig = plt.figure(figsize=(4,2))
ax = fig.add_axes([0,0,1,1])
#ax.bar(['Bicycle','No Bicycle'], [len(crowd_data_set[crowd_data_set['bicycle?']==True]),len(crowd_data_set[crowd_data_set['bicycle?']==False])])
b1 = ax.bar(['Bicycle'], c_is_bicycle)
b2 = ax.bar('No Bicycle', c_is_not_bycle)
ax.set_xlabel('Class label')
ax.set_ylabel('Number of samples')
ax.set_title('Class distribution in dataset')
plt.show()

# Task 4

In [ ]:
# add image name
crowddataframe['img'] = crowddataframe['root_input.image_url'].apply(lambda  s: s[-12:-4])
# add truth to dataset
crowddata_truth = crowddataframe.merge(references, on='img', how='left')
crowddata_truth.tail(2)

In [ ]:
crowddata_truth['answer_bool'] =crowddata_truth['task_output.answer']!='no'

In [ ]:
# wrong labeled

crowddata_truth['correct_annotation'] =  crowddata_truth['answer_bool'] == crowddata_truth['is_bicycle']

In [ ]:
crowddata_truth.head(3)

In [ ]:
crowddata_truth.groupby('user.vendor_user_id').count()

In [ ]:
# 
annotation_count = crowddata_truth.groupby('user.vendor_user_id')['correct_annotation'].agg(['count', 'sum']).reset_index()
annotation_count['relativ'] = annotation_count['sum'] / annotation_count['count']
annotation_count = annotation_count.sort_values('relativ')

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_axes([0,0,1,1])

annotator_nr = annotation_count['user.vendor_user_id'].apply(lambda x: x[-2:])

ax.scatter(annotator_nr,annotation_count['relativ'])

ax.set_xlabel('Annotator')
ax.set_ylabel('% correct')
ax.set_title('Annotation-Quality')